### Nuißl Sandra, 14.08.2023
## "Empirische Evaluation von ‚State Of The Art‘ Topic Modeling Ansätze am Beispiel von Produktreviews für die Entscheidungsunterstützung in Unternehmen"
### - Beschleunigung der Data Preperation mithilfe von Pyspark -
![Step1](https://github.com/Sannui/Masterarbeit_Nuissl_Sandra/blob/main/Titelbild.jpg)
<center><img src="https://github.com/Sannui/Masterarbeit_Nuissl_Sandra/blob/main/Titelbild.jpg" height="300px" width="1100px"/></center>
<center><font size="1">https://www.cloudways.com/blog/wp-content/uploads/Product-Review-1024x576.jpg</font></center>


<hr>

## **Inhaltsverzeichnis**

<ul>1. Aufbau des Jupyter Notebooks</ul>
<ul>2. Instalation und Imports</ul>
    <ul>
     <ul>2.1. Installationen</ul>
     <ul>2.2. Imports</ul>
    </ul>
<ul>3. Laden der Amazon Daten</ul>
    <ul>
     <ul>3.1. Entpacken der Zip Files</ul>
     <ul>3.2. Laden des Datensatzes in einen Data Frame</ul>
    </ul>
<ul>4. Data Preperation</ul>
    <ul>
     <ul>4.1. Lowercasing</ul>
     <ul>4.2. Lemmatisierung</ul>
     <ul>4.3. Stemming</ul>
     <ul>4.4. Entfernung von Satzzeichen</ul>
     <ul>4.5. Entfernung von Stopwords</ul>
     <ul>4.6. Entfernung von Zahlen</ul>
     <ul>4.7. Entfernung von nicht ASCII konformen Wörtern</ul>
     <ul>4.8. Entfernung von Links</ul>
     <ul>4.9. Tokenization</ul>
    </ul>
<ul>5. Stoppen der Spark seccion</ul>
<ul>6. Literaturverzeichnis</ul>
<hr>

## 1. Aufbau des Jupyter Notebooks
Im Rahmen dieser Masterarbeit wurde versucht, die Beschleunigung der Data Preperation mithilfe von Pyspark zu implementieren, um die Perfomance zu verbessern und die Ladevorgänge zu beschleunigen. 

PySpark wurde von den Entwicklern des Big-Data-Systems Apache Spark herausgebracht und ist eine Version, welche in der Python Programmierung Anwendung findet. Die Architektur verteilt sich auf ein Cluster, wodurch es ermöglicht wird große Datenmengen zu parallelisieren und performanter zu verarbeiten. Hierfür wird auf den Arbeitsspeicher der Hardware zugegriffen wodurch die Festplatte nicht belastet wird (Wuttke, Einführung in Apache Spark: Komponenten, Vorteile und Anwendungsbereiche, 2022). Es gilt als eines der besten Frameworks in der Datenwissenschaft und verbindet Big Data mit maschinellem Lernen, da es sowohl In-Memory-Operationen für eine bis zu 100-fache Beschleunigung liefert als auch Zusatzpakete wie MLib und GraphX bietet. Spark läuft auf einer Virtuellen Maschine von Java (JVM), ist in Scala geschrieben und auf Hadoop/HDFS implementiert. PySpark ist die Python-API, welche eine Schnittstelle zwischen der Python-Programmierung und dem Spark-Framework liefert. Auf diese Weise ist es möglich mithilfe von Objekten die Vorteile von Spark zu nutzen, ohne die Programmiersprache Scala zu erlernen (Sakar, 2018).

Da jedoch für das Anzeigen von Graphiken und das speichern des verarbeiteten Datensatzes die Cloudumgebung von Databricks benötigt wird, wurde dieser Ansatz wieder verworfen. Der vollständigkeit halber ist in diesem Notebook die explorative Auseinandersetzung mit der Data Preperation mithilfe von Pyspark festgehalten.

## 2. Instalationen und Imports

Um mit Apache Spark in Juyter Notebooks arbeiten zu können, muss die richtige Version zuerst heruntergeladen werden. Hierfür kann auf der Webside von Apache Spark die richtige Version heruntergeladen werden.

Darüber hinaus ist es erforderlich Java (JDK) zu installieren. Benötigt wird die Version 8, damit Python in der Lage ist mit der Virtuellen Maschine von Java zu kommunizieren (Preusler, 2020). Für den Download kann direkt die Website von Java aufgerufen werden (Oracle, 2023).
Nach dem Download befindet sich die Datei „JavaSetup8u351.exe“ im Download Ordner. Diese muss lediglich in den aktuellen Arbeitsordner kopiert werden. Es ist wichtig, dass sich die Downloads von Spark und Java in dem gleichen Ordner befinden, in welchem das Jupyter Notebook abgelegt ist, damit das Coding fehlerfrei läuft. Nach den genannten Korrekturen konnte die Spark Session erfolgreich gestartet werden.


### 2.1. Installationen

Nun kann die Library über den „pip install“ Befehl oder über das Terminal installiert werden. In diesem Fall hat die Installation über das Terminal stattgefunden (Shah, 2018).

In [ ]:
# Instalationen
% pip install gzip
% pip install shutil
% pip install findspark
% pip install pyspark
% pip install nltk

### 2.2. Imports
Nach der erfolgreichen Installation von pyspark können die für die Beschleunigung des Ladevorgang benötigten Klassen importiert werden. Wichtig ist jedoch, dass zuvor „findspark“ importiert und initialisiert wurde, damit die benötigte Spark Instanz gewunden wird (Shah, 2018).

In [2]:
# Imports und Initalisierungen
import pandas as pd

# Progressbar
from tqdm.auto import tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

# Entpacken der Files
import gzip
import shutil

# Beschleunigung mit Joblib
import json
import joblib
from joblib import Parallel, delayed

# Beschleunigung mit Pyspark
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, split
from pyspark.sql import functions as f
from pyspark.sql.types import StringType, ArrayType

# Datenverarbeitung
import re

# Natural Language Processing
import nltk
from nltk.stem.wordnet import WordNetLemmatizer             # Lemmatisierung zur Textdimensionsreduktion
nltk.download('averaged_perceptron_tagger')                 # Für POS-Tagging
from nltk import word_tokenize                              # Tokenisierung
from nltk import pos_tag                                    # Bestimmung der grammatikalischen Token
nltk.download('stopwords')                                  # Herunterladen der Liste mit Stopwords
stopwords.words("english")                                  # Sprache der Stopwords
from nltk.stem.snowball import SnowballStemmer              # Stemmer zur Textdimensionsreduktion
from nltk.corpus import stopwords                           # Zur entfernung der Stopwords

## 3. Laden der Amazon Daten

### 3.1. Entzippen der Json Files
Die Amazon Datensätze sind aufgrund der großen Datenmengen als Zip Dateien gespeichert. Um diese in das Jupyter Notebook einlesen zu können, müssen daher die JSON Files zuerst entpackt werden.

Zum Entpacken der Files wird im Folgenden "gzip" in Verbindung mit "shutil" verwendet. 
Gzip ist ein Programm zur Daten Kompression (Free Software Foundation, 2022) während shutil ein Modul ist, welches diverse High-Level-Operationen zur Unterstützung beim Kopieren und Löschen von Dateien bietet (Python-Software-Foundation, 2023). 
Durch deren Kombination werden zuerst zwei Files geöffnet. file_in beschreibt hier das gezippte JSON file und bei file_out handelt es sich um ein leeres JSON File, in welches die Daten aus file_in mithilfe der Funktion "copyfileobj" von shutil kopiert werden (Erick, 2018). 

In [30]:
# Entpacken der Datei und speichern in einem JSON File
# Quelle: https://stackoverflow.com/questions/31028815/how-to-unzip-gz-file-using-python
# Review Daten
with gzip.open('Sports_and_Outdoors.json.gz', 'rb') as file_in:
    with open('Sports_and_Outdoors.json', 'wb') as file_out:
        shutil.copyfileobj(file_in, file_out)

# Meta Daten
with gzip.open('meta_Sports_and_Outdoors.json.gz', 'rb') as file_in:
    with open('meta_Sports_and_Outdoors.json', 'wb') as file_out:
        shutil.copyfileobj(file_in, file_out)

### 3.2. Laden des Datensatzes in einen Data Frame

Nach dem Erfolgreichen Abschluss aller Installationen und Imports, sowie dem Entpacken der Files, kann als nächstes eine Spark Session mit folgendem Befehl gestartet werden (Shah, 2018):

In [3]:
# Starten der Spark Session
# https://medium.com/@ashish1512/how-to-setup-apache-spark-pyspark-on-jupyter-ipython-notebook-3330543ab307
spark = SparkSession.builder.getOrCreate()
spark

Aufgrund der verschachtelten Struktur des Datensatzes kommt es dazu, dass Duplikate in den Spaltennamen vorhanden sind. Dies ist der Fall, da mehrere Spalten mit dem Wort „style“ beginnen. Um diesen Fehler zu umgehen, kann die Spark Session konfiguriert werden, indem die Berücksichtigung der Groß- und Kleinschreibung aktiviert wird (shoeboxer, 2015):

In [4]:
# Konfiguration "Case Sensitive" zur Normalisierung
# Quelle: https://stackoverflow.com/questions/33816481/duplicate-columns-in-spark-dataframe
spark.conf.set("spark.sql.caseSensitive", "true")

Mit der folgenden Funktion wird der Datensatz in eine Data Frame laden. Durch die Methode "format" lässt sich die Datenquelle der Datei weiter spezifizieren. (SparkBy{Examples}, 2020).

In [6]:
# Laden des Json Files  der Meta Data in einen Spark Data Frame
# Quelle: https://sparkbyexamples.com/pyspark/pyspark-read-json-file-into-dataframe/#read-json-multiline?utm_content=cmp-true
meta_df = spark.read.format('org.apache.spark.sql.json').load("meta_Sports_and_Outdoors.json")

Nachdem die Meta Daten geladen wurden, werden nun die Review Daten eingespielt. Um die Auswirkungen der Beschleunigung besser darstellen zu können, wurde der Ladevorgang mit Joblib wiederholt. Die Progressbars zeigen einen deutlichen Unterschied der Geschwindigkeit der Ladevorgänge.

In [6]:
# Laden der Review Daten mithilfe von Pyspark
# Quelle: https://sparkbyexamples.com/pyspark/pyspark-read-json-file-into-dataframe/#read-json-multiline?utm_content=cmp-true
for i in tqdm(range(1), desc ="Laden von 12.980.837 Zeilen mithilfe von Pyspark: "):
    review_df = spark.read.format('org.apache.spark.sql.json').load("Sports_and_Outdoors.json")

# Laden der Review Daten mithilfe von Joblib
r = open('Sports_and_Outdoors.json')
dataset_review = Parallel(n_jobs=-1)(delayed(json.loads)(line) for line in tqdm(r,
                                                                                desc ="Laden von 12.980.837 Zeilen mithilfe von Joblib: ",
                                                                                total = 12980837))

Laden von 12.980.837 Zeilen mithilfe von Pyspark:   0%|          | 0/1 [00:00<?, ?it/s]

Laden von 12.980.837 Zeilen mithilfe von Joblib:   0%|          | 0/12980837 [00:00<?, ?it/s]

Laden von 12.980.837 Zeilen mithilfe von Joblib:   0%|          | 0/12980837 [00:00<?, ?it/s]

## 4. Datapreparation

Um einen Konsistenten Datensatz für die weitere Verarbeitung und die Anwendung der Topic Modelling Modelle zu erhalten, werden die oben aufgeführten Bereinigungen durchgeführt.

Doch befor die Textbereinigung stattfindet, werden alle Zeilen, welche in den reviewText keine Werte enthalten gelöscht.
Darüber hinaus werden die Spalten des Data Frame reduziert, sodass lediglich diese Spalten für die Modellierung benötigt werden und die Performance durch die verringerung der Daten verbessert werden kann.

In [10]:
# Neuer Data Frame, welcher die relevanten Spalten in englischer Sprache beinhaltet
text_df = review_df.select("reviewText", "reviewTime", "asin", "overall")

# Herausfiltern der leeren "non-type values" aus den Data Frame (wird hier nicht benötigt)
text_df = text_df.where(text_df.reviewText.isNotNull())

### 4.1. Lowercasing

Mithilfe des Lowercasings werden alle Großbuchstaben in Kleinbuchstaben konvertiert, da diese für das Verständnis der Texte nicht benötigt werden. Hierfür wird zuerst eine Funktion definiert, welche einen string in Kleinbuchstaben konvertiert (Pomer, 2022).

Im Anschluss wird mithilfe von PySpark die Funktion auf den Data Frame angewendet und die bereinigten Ergebnisse in einer neuen Spalte gespeichert.
Abschließend werden alle "non-type-values" aus dem Datensatz herausgefilter, bevor dieser den nächsten Bereinigungsschritt durchläuft (Yuzhe-Lu, 2018).
Das Ergebnis des Lowercasing lässt sich in der Ausgabe nachverfolgen.

In [11]:
# Definition der Funktion, um alle Buchstaben in Kleinschreibung zu konvertieren
# Quelle: https://thecattlecrew.net/2022/08/03/textklassifikation-vorverarbeitung-der-daten/
def to_lower(in_string):
    out_string = in_string.lower()
    return out_string

In [13]:
# Anwenden der Funktion auf den Datensatz
# Quelle: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb

# setup PySpark udf Funktion
to_lower_udf = udf(to_lower, StringType())

# Ergänzen des Data Frames mit Spalte mit beneinigten Daten
text_df = text_df.withColumn("text_lower", to_lower_udf(text_df["reviewText"]))

# Herausfiltern der leeren "non-type values" aus den Data Frame (wird hier nicht benötigt)
text_df = text_df.where(text_df.text_lower.isNotNull())

# Ausgabe (Vergleich des unbereinigten und bereinigten Daten)
print(text_df.select("reviewText", "text_lower").take(2))

[Row(reviewText='What a spectacular tutu! Very slimming.', text_lower='what a spectacular tutu! very slimming.'), Row(reviewText='What the heck? Is this a tutu for nuns? I know you can cut it but STILL. Also there aren\'t several layers of the tutu making it "poof out" It just lays flat. Needless to say it was returned.', text_lower='what the heck? is this a tutu for nuns? i know you can cut it but still. also there aren\'t several layers of the tutu making it "poof out" it just lays flat. needless to say it was returned.')]


### 4.2. Lemmatisierung
Bei der Lemmatisierung handelt es sich um die Umwandlung von Wörtern in ihre Grundform unter Verwendung von Wörterbüchern (Pomer, 2022). Hierbei werden Pluralformen in Singularformen und unterschiedliche Zeitformen in das Präsenz umgewandelt (Yuzhe-Lu, 2018). Für die Lemmatisierung wird eine Funktion definiert, welche die Sätze in einzelen Token aufsplittet, desse POS-Tags ermittelt und basierend dieser Tags die Lemmatisierung mithilfe der Funktion "lemmatize()" durchführt (Johnson, 2023).  

Im Anschluss wird mithilfe von PySpark die Funktion auf den Data Frame angewendet und die bereinigten Ergebnisse in einer neuen Spalte gespeichert.
Abschließend werden alle "non-type-values" aus dem Datensatz herausgefilter, bevor dieser den nächsten Bereinigungsschritt durchläuft (Yuzhe-Lu, 2018).
Das Ergebnis der Lemmatisierung lässt sich in der Ausgabe nachverfolgen.

In [14]:
# Definition der Funktion zur Lemmatisierung, um Wörter in ihre Grundform zu konvertieren
# Quelle 1: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb
# Quelle 2: https://www.machinelearningplus.com/nlp/lemmatization-examples-python/#stanfordcorenlplemmatization

def lemmatize(in_string):
    # Definition der notwendigen Parameter
    list_pos = 0                                  # Zuordnung einer Positionsnummer
    cleaned_str = ''                              # Leerer String für bereinigte Wörter
    text_token = nltk.word_tokenize(in_string)    # Tokenisieren der Sätze in einzelne Strings
    tagged_words = pos_tag(text_token)            # Grammatikalisches Tagging
    wnl = WordNetLemmatizer()                     # Klasse von NLTK für Lemmatisierung
    
    # Durchfürhung der Lemmatisation und Zusammenführung der Ergebnisse in einen String
    for word in tagged_words:
        if 'v' in word[1].lower():
            lemma = wnl.lemmatize(word[0], pos='v')
        else:
            lemma = wnl.lemmatize(word[0], pos='n')
        if list_pos == 0:
            cleaned_str = lemma
        else:
            cleaned_str = cleaned_str + ' ' + lemma
        list_pos += 1
    return cleaned_str

In [15]:
# Anwenden der Funktion auf den Datensatz
# Quelle: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb

# setup PySpark udf Funktion
lemmatize_udf = udf(lemmatize, StringType())

# Ergänzen des Data Frames mit Spalte mit beneinigten Daten
text_df = text_df.withColumn("text_lemmatize", lemmatize_udf(text_df["text_lower"]))

# Herausfiltern der leeren "non-type values" aus den Data Frame
text_df = text_df.where(text_df.text_lemmatize.isNotNull())

# Ausgabe (Vergleich des unbereinigten und bereinigten Daten)
print(text_df.select("text_lower", "text_lemmatize").take(2))

[Row(text_lower='what a spectacular tutu! very slimming.', text_lemmatize='what a spectacular tutu ! very slimming .'), Row(text_lower='what the heck? is this a tutu for nuns? i know you can cut it but still. also there aren\'t several layers of the tutu making it "poof out" it just lays flat. needless to say it was returned.', text_lemmatize="what the heck ? be this a tutu for nun ? i know you can cut it but still . also there be n't several layer of the tutu make it `` poof out '' it just lay flat . needle to say it be return .")]


### 4.3. Stemming

Da die Lemmatisierung nicht alle Worte bereinigt hat, wie es zum Beispiel bei dem Wort "quickly" zu erkennen ist, welches eigentlich in "quick" konvertiert werden sollte, wird im folgenden zusätzlich der sogenannte Snowball-Stemming-Algorithmus implementiert.

Im Gegensatz zur Lemmatisierung, beruft sich das Stemming nicht auf Wörterbücher, sondern entfernt die Sufixe und Präfixe der Wörter. Es findet daher kein Bezug auf den Kontext der Wörter statt
(Luber & Litzel, Was ist Stemming?, 2020).

Zur Durchführung des Stemming wird eine Funktion definiert, welche die Sätze zuerst in einzelne Token unterteilt und den Snowball Stemmer auf diese mithilfe einer For-Schleife anwendet und wieder zu einem Satz verbindet (NLTK Project, 2023).
Im Anschluss wird mithilfe von PySpark die Funktion auf den Data Frame angewendet und die bereinigten Ergebnisse in einer neuen Spalte gespeichert. Darüber hinaus werden alle "non-type-values" aus dem Datensatz herausgefilter, bevor dieser den nächsten Bereinigungsschritt durchläuft (Yuzhe-Lu, 2018).
Das Ergebnis des Stemmings lässt sich in der Ausgabe nachverfolgen.

In [16]:
# Definition der Funktion zur Durchführung des Stemmings
# Quelle: https://www.nltk.org/howto/stem.html
def stemming(in_string):
    # Definition der notwendigen Parameter
    text_token = nltk.word_tokenize(in_string)      # Tokenisieren der Sätze in einzelne Strings
    stemmer = SnowballStemmer("english")            # Laden der Klasse zum Stemming
    
    # Durchfürhung des Stemmings und Zusammenführung der Ergebnisse in einen String
    cleaned_str = ' '.join([stemmer.stem(word) for word in text_token])
    return cleaned_str

In [18]:
# Anwenden der Funktion auf den Datensatz
# Quelle: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb

# setup PySpark udf Funktion
stemming_udf = udf(stemming, StringType())

# Ergänzen des Data Frames mit Spalte mit beneinigten Daten
text_df = text_df.withColumn("text_stemming", stemming_udf(text_df["text_lemmatize"]))

# Herausfiltern der leeren "non-type values" aus den Data Frame
text_df = text_df.where(text_df.text_stemming.isNotNull())

# Ausgabe (Vergleich des unbereinigten und bereinigten Daten)
print(text_df.select("text_lemmatize", "text_stemming").take(2))

[Row(text_lemmatize='what a spectacular tutu ! very slimming .', text_stemming='what a spectacular tutu ! veri slim .'), Row(text_lemmatize="what the heck ? be this a tutu for nun ? i know you can cut it but still . also there be n't several layer of the tutu make it `` poof out '' it just lay flat . needle to say it be return .", text_stemming="what the heck ? be this a tutu for nun ? i know you can cut it but still . also there be n't sever layer of the tutu make it `` poof out `` it just lay flat . needl to say it be return .")]


### 4.4. Entfernung von Satzzeichen

Nachdem die bereinigung der Wörter selbst abgeschlossen ist, werden nun die Satzzeichen aus dem Datensatz entfernt. Viele Satzzeichen verändern bei ihrem Fehlen den Kontext nicht groß, sodass diese problemlos bereinigt werden können. Jedoch ist zu beachten, dass beispielsweise Kommata oder Fragezeichen den Inhalt eines Textes verändern können, sodass es bei der Analyse zu Fehlinterpretationen kommt. Besonders zu beachten sind in diesem Zusammenhang Zahlen, da es bei dem Jahr "1998" und einem Geldwert "19,98" einen großen Unterschied macht, ob das Komma im Datensatz erhalten bleibt oder ob es verschwindet. Bei letzteren Fall hätten beide Zahlen den identischen Wert und das Ergebnis der Auswertung hätte keine Aussagekraft (Pomer, 2022). Aus diesem Grund soll im Folgenden mit diesem Wissen im Hinterkopf eine Bereinigung der Satzzeichen stattfinden. Treten bei dem Ergebnis der Analyse vermehrt Zahlen oder andere Unstimmigkeiten auf, müssen die Satzzeichen ggf. in die Modelle miteinbezogen werden.

Für die Bereinigung werden folgende Zeichen durch einen leeren Wert mithilfe der PySpark functions "col()" und translate()" ersetzt. Der Import der functions ist in Abschnitt 1. Installationen und Imports zu finden (Apache Spark, kein Datum):

> '!"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~'

Abschließend werden alle "non-type-values" aus dem Datensatz herausgefilter, bevor dieser den nächsten Bereinigungsschritt durchläuft (Yuzhe-Lu, 2018).
Das Ergebnis für die Entfernung der Satzzeichen lässt sich in der Ausgabe nachverfolgen.

In [20]:
# Entfernung der Satzzeichen mit Hilfe von PySpark
# Quelle: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html
text_df = (text_df.withColumn("text_punctation", f.translate(f.col("text_stemming"), '!"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~', '')))

# Herausfiltern der leeren "non-type values" aus den Data Frame
text_df = text_df.where(text_df.text_punctation.isNotNull())

# Ausgabe (Vergleich des unbereinigten und bereinigten Daten)
print(text_df.select("text_stemming", "text_punctation").take(2))

[Row(text_stemming='what a spectacular tutu ! veri slim .', text_punctation='what a spectacular tutu  veri slim '), Row(text_stemming="what the heck ? be this a tutu for nun ? i know you can cut it but still . also there be n't sever layer of the tutu make it `` poof out `` it just lay flat . needl to say it be return .", text_punctation='what the heck  be this a tutu for nun  i know you can cut it but still  also there be nt sever layer of the tutu make it `` poof out `` it just lay flat  needl to say it be return ')]


### 4.5. Entfernung von Stopwords

Stoppwörter können in Vorbereitung auf das Topic Modelling ebenfalls herausgefiltert werden, da diese keinen Einfluss auf die Bedeutung des Kontextes haben. Aufgrund ihrer Häufigkeit in einem Text wird darüber hinaus der Datensatz drastisch reduziert, wordurch die Laufzeit sowie die Genauigkeit der Analyse verbessert werden kann. Die Auswahl der Stopword muss jedoch bedacht getroffen werden, um die Ergebnisse nicht zu verfälschen. So ist beispielsweise die Negation "nicht" in vielen Sprachen als Stopword eingestuft, kann jedoch den Kontext eines Satzes bei dessen Entfernung dramatisch verändern (Teja, 2020). Für jede Sprache existieren unterschiedliche Stoppwörter. Um diese aus den Daten herauszufiltern werden sogenannte Stoppwordlisten herangezogen (Pomer, 2022).

Für die Bereinigung der Daten im Rahmen dieser Masterarbeit wird eine Liste der englischen Stopword der NLTK Library verwendet. 
Zur Durchführung der Bereinigung wird eine Funktion definiert, welche die Token des Amazon Datensatzes mit der Liste der Stopwords vergleicht und einen bereinigten String kreiert, welcher keine Stopwords mehr enthält (Yuzhe-Lu, 2018).

Im Anschluss wird mithilfe von PySpark die Funktion auf den Data Frame angewendet und die bereinigten Ergebnisse in einer neuen Spalte gespeichert (Johnson, 2023).
Abschließend werden alle "non-type-values" aus dem Datensatz herausgefilter, bevor dieser den nächsten Bereinigungsschritt durchläuft (Yuzhe-Lu, 2018).
Das Ergebnis für die Entfernung der Satzzeichen lässt sich in der Ausgabe nachverfolgen.

In [24]:
# Definition der Funktion zur Entfernung der Stopwords
# Quelle: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb
def remove_stops(in_string):
    # Definition der notwendigen Parameter
    list_pos = 0                                  # Zuordnung einer Positionsnummer
    cleaned_str = ''                              # Leerer String für bereinigte Wörter
    text_token = nltk.word_tokenize(in_string)    # Tokenisieren der Sätze in einzelne Strings
    stop_words = stopwords.words('english')       # Bestimmen der Stopwords (für Englisch)
    stop_words.append('would')                    # Hinzufügen individueller Wörter zur Liste der Stopwords

    # Durchfürhung der Entfernung der Stopwords und Zusammenführung der Ergebnisse in einen String
    for word in text_token:
        if word not in stop_words:
            if list_pos == 0:
                cleaned_str = word
            else:
                cleaned_str = cleaned_str + ' ' + word
            list_pos += 1
    return cleaned_str

In [25]:
# Anwenden der Funktion auf den Datensatz
# Quelle: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb
# Quelle: https://www.legendu.net/misc/blog/spark-issue:-_pickle.PicklingError:-args[0]-from-__newobj__-args-has-the-wrong-class/

# setup PySpark udf Funktion
remove_stops_udf = udf(remove_stops, StringType())

# Ergänzen des Data Frames mit Spalte mit beneinigten Daten
text_df = text_df.withColumn("text_stopwords", remove_stops_udf(text_df["text_punctation"]))

# Herausfiltern der leeren "non-type values" aus den Data Frame
text_df = text_df.where(text_df.text_stopwords.isNotNull())

# Ausgabe (Vergleich des unbereinigten und bereinigten Daten)
print(text_df.select("text_punctation", "text_stopwords").take(2))

[Row(text_punctation='what a spectacular tutu  veri slim ', text_stopwords='spectacular tutu veri slim'), Row(text_punctation='what the heck  be this a tutu for nun  i know you can cut it but still  also there be nt sever layer of the tutu make it `` poof out `` it just lay flat  needl to say it be return ', text_stopwords='heck tutu nun know cut still also nt sever layer tutu make `` poof `` lay flat needl say return')]


### 4.6. Entfernung von Zahlen

Im Rahmen der Masterarbeit wurde beschlossen, die Zahlen aus dem Datensatz zu entfernen, da die Annahme getroffen wurde, dass diese nicht zur Einteilung in die richtigen Kategorien beitragen.
Mithilfe der Regular expressions werden die Nummern 0 bis 9 in einerm Parameter gespeichert. Dieser wird daraufhin mit dem Input Text abgeglichen und bei einer Übereinstimmung wird die Zahl mit "nichts" ausgetauscht (Python Software Foundation, 2023). 

Im Anschluss wird mithilfe von PySpark die Funktion auf den Data Frame angewendet und die bereinigten Ergebnisse in einer neuen Spalte gespeichert (Johnson, 2023).
Abschließend werden alle "non-type-values" aus dem Datensatz herausgefilter, bevor dieser den nächsten Bereinigungsschritt durchläuft (Yuzhe-Lu, 2018).
Das Ergebnis für die Entfernung der Satzzeichen lässt sich in der Ausgabe nachverfolgen.

In [26]:
# Definition der Funktion zur Entfernung der Zahlen
# Quelle: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb
# Quelle: https://thecattlecrew.net/2022/08/03/textklassifikation-vorverarbeitung-der-daten/
def remove_numbers(in_string):
    # Kompilieren der Zahlen 0-9 in ein Ausdrucksmuster
    num_re = re.compile('(\\d+)')

    # Ersetzen der Zahlen durch "nichts"
    cleaned_str = " ".join((re.sub(num_re, "", in_string)).split()) 
    return cleaned_str

In [28]:
# Anwenden der Funktion auf den Datensatz
# Quelle: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb

# setup PySpark udf Funktion
remove_numbers_udf = udf(remove_numbers, StringType())

# Ergänzen des Data Frames mit Spalte mit beneinigten Daten
text_df = text_df.withColumn("text_numbers", remove_numbers_udf(text_df["text_stopwords"]))

# Herausfiltern der leeren "non-type values" aus den Data Frame
text_df = text_df.where(text_df.text_numbers.isNotNull())

# Ausgabe (Vergleich des unbereinigten und bereinigten Daten)
print(text_df.select("text_stopwords", "text_numbers").take(2))

[Row(text_stopwords='spectacular tutu veri slim', text_numbers='spectacular tutu veri slim'), Row(text_stopwords='heck tutu nun know cut still also nt sever layer tutu make `` poof `` lay flat needl say return', text_numbers='heck tutu nun know cut still also nt sever layer tutu make `` poof `` lay flat needl say return')]


### 4.7. Entfernung von nicht ASCII konvormen Wörtern

ASCII ist die Kurzform für „American Standard Code for Information Interchange“ und dient zur standartisierten Darstellung von Zeichen in elektronischer Form (IONOS SE, 2022). Zu beachten ist, dass es bei diesem um einen amerikanischen Standart handelt und daher keine Sonderzeichen, wie „ß“ oder „é, á“ berücksichtigt und daher beispielsweise bei deutschen Datensätzen das ergebnis verfälschen könnte (Pomer, 2022). Da der Amazondatensatz zufor auf die Sprache geprüft und bereinigt wurde, beinhaltet dieser lediglich englische Sätze, wodurch der ASCII Standard verwendet werden kann.

Zur Bereinigung wird die untenstehende Funktion definiert, welche den Unicode der wörter ermittelt und diese mit den Unicodes aus dem ASCII Standard abgleicht. Lediglich die Wörter, welche zwischen 0 und 128 liegen werden im Datensatz beibehalten.
Im Anschluss wird mithilfe von PySpark die Funktion auf den Data Frame angewendet und die bereinigten Ergebnisse in einer neuen Spalte gespeichert (Johnson, 2023).
Abschließend werden alle "non-type-values" aus dem Datensatz herausgefilter, bevor dieser den nächsten Bereinigungsschritt durchläuft (Yuzhe-Lu, 2018).
Das Ergebnis für die Entfernung der Satzzeichen lässt sich in der Ausgabe nachverfolgen.

In [29]:
# Definition der Funktion zur Entfernung der ASCII characters
# Quelle: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb
def remove_non_ascii_words(in_string):
    ''' Returns the string without non ASCII characters'''
    cleaned_str = ''.join(word for word in in_string if 0 < ord(word) < 128)
    return cleaned_str

In [31]:
# Anwenden der Funktion auf den Datensatz
# Quelle: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb

# setup PySpark udf Funktion
remove_non_ascii_words_udf = udf(remove_non_ascii_words, StringType())

# Ergänzen des Data Frames mit Spalte mit beneinigten Daten
text_df = text_df.withColumn("text_ascII", remove_non_ascii_words_udf(text_df["text_numbers"]))

# Herausfiltern der leeren "non-type values" aus den Data Frame
text_df = text_df.where(text_df.text_ascII.isNotNull())

# Ausgabe (Vergleich des unbereinigten und bereinigten Daten)
print(text_df.select("text_numbers", "text_ascII").take(2))

[Row(text_numbers='spectacular tutu veri slim', text_ascII='spectacular tutu veri slim'), Row(text_numbers='heck tutu nun know cut still also nt sever layer tutu make `` poof `` lay flat needl say return', text_ascII='heck tutu nun know cut still also nt sever layer tutu make `` poof `` lay flat needl say return')]


### 4.8. Entfernung von Webside Links und Mentions

Im Anschluss wird mithilfe von PySpark die Funktion auf den Data Frame angewendet und die bereinigten Ergebnisse in einer neuen Spalte gespeichert (Johnson, 2023).
Abschließend werden alle "non-type-values" aus dem Datensatz herausgefilter, bevor dieser den nächsten Bereinigungsschritt durchläuft (Yuzhe-Lu, 2018).
Das Ergebnis für die Entfernung der Satzzeichen lässt sich in der Ausgabe nachverfolgen.

In [32]:
# Definition der Funktion zur Entfernung der Hyperlinks, html-Symbolden und Erwähnungen
# Quelle: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb
def remove_features(in_string):
    # Festlegung der Bedingungen
    url_re = re.compile('https?://(www.)?\w+\.\w+(/\w+)*/?')
    html_re = re.compile("<br />")
    mention_re = re.compile('@(\w+)')

    # Ersetzen der definierten Bedingungen durch " "
    cleaned_string = url_re.sub(' ', in_string)         # Entfernung von Hyperlinks
    cleaned_string = html_re.sub(' ', in_string)        # Entfernung der html Symbole    
    cleaned_string = mention_re.sub(' ', in_string)     # Rentfernen von Erwähnungen mit "@mentions"

    return cleaned_string

In [33]:
# Anwenden der Funktion auf den Datensatz
# Quelle: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb

# setup PySpark udf Funktion
remove_features_udf = udf(remove_features, StringType())

# Ergänzen des Data Frames mit Spalte mit beneinigten Daten
text_df = text_df.withColumn("text_features", remove_features_udf(text_df["text_ascII"]))

# Herausfiltern der leeren "non-type values" aus den Data Frame
text_df = text_df.where(text_df.text_features.isNotNull())

# Ausgabe (Vergleich des unbereinigten und bereinigten Daten)
print(text_df.select("text_numbers", "text_ascII").take(2))

[Row(text_numbers='spectacular tutu veri slim', text_ascII='spectacular tutu veri slim'), Row(text_numbers='heck tutu nun know cut still also nt sever layer tutu make `` poof `` lay flat needl say return', text_ascII='heck tutu nun know cut still also nt sever layer tutu make `` poof `` lay flat needl say return')]


### 4.9. Tokenize

In [34]:
# Anwenden des Tokenizers auf den DataFrame
# Quelle: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb

# setup PySpark udf Funktion
tokenize_udf = udf(word_tokenize, ArrayType(StringType()))

# Ergänzen des Data Frames mit Spalte mit beneinigten Daten
text_df = text_df.withColumn("token_text", tokenize_udf(text_df["text_features"]))

# Herausfiltern der leeren "non-type values" aus den Data Frame
text_df = text_df.where(text_df.token_text.isNotNull())

# Ausgabe (Vergleich des unbereinigten und bereinigten Daten)
print(text_df.select("text_features", "token_text").take(2))

[Row(text_features='spectacular tutu veri slim', token_text=['spectacular', 'tutu', 'veri', 'slim']), Row(text_features='heck tutu nun know cut still also nt sever layer tutu make `` poof `` lay flat needl say return', token_text=['heck', 'tutu', 'nun', 'know', 'cut', 'still', 'also', 'nt', 'sever', 'layer', 'tutu', 'make', '``', 'poof', '``', 'lay', 'flat', 'needl', 'say', 'return'])]


### 4.10. Merge mit Metadaten und Selection der Spalten


In [38]:
# Merge der Meta Daten mit dem bereinigeten Datensatz
merged_df = text_df.join(meta_df, text_df.asin == meta_df.asin, 'left')

# Neuen Data Frame erzeugen, welcher nur die relevanten Spalten enhält
cleaned_data_df = merged_df.select("token_text", "reviewTime", "overall", "brand", "price", "title")

## 5. Stoppe der Spark Session

In [40]:
# Spark Session deaktivieren
spark.stop()

## 6. Literaturverteichnis

Apache Spark. (17. 02 2023). Download Apache Spark™. Abgerufen am 20. 02 2023 von spark.apache.org: https://spark.apache.org/downloads.html

Erick. (12. 02 2018). How to unzip gz file using Python. Abgerufen am 24. 10 2022 von stackoverflow.com: https://stackoverflow.com/questions/31028815/how-to-unzip-gz-file-using-python

Free Software Foundation, I. (02. 04 2022). GNU Gzip: General file (de)compression. Abgerufen am 20. 02 2023 von gnu.org: https://www.gnu.org/software/gzip/manual/gzip.html

IONOS SE. (05. 04 2022). ASCII – Erklärung und Beispiele. Abgerufen am 08. 03 2023 von ionos.de: https://www.ionos.de/digitalguide/server/knowhow/ascii-american-standard-code-for-information-interchange/

Johnson, D. (21. 01 2023). POS-Tagging mit NLTK und Chunking in NLP [BEISPIELE]. Abgerufen am 21. 02 2023 von guru99.com: https://www.guru99.com/pos-tagging-chunking-nltk.html#1

Luber, D.-I. & Litzel, N. (30. 11 2020). Was ist Stemming? Abgerufen am 22. 02 2023 von bigdata-insider.de: https://www.bigdata-insider.de/was-ist-stemming-a-980852/#:~:text=Abgrenzung%20von%20Stemming%20und%20Lemmatisierung&text=Stemming%2DAlgorithmen%20arbeiten%20meist%20mit,)%2C%20um%20Stammformen%20zu%20finden.

NLTK Project. (02. 01 2023). Natural Language Toolkit - Sample usage for stem. Abgerufen am 22. 02 2023 von nltk.org: https://www.nltk.org/howto/stem.html

Oracle. (17. 01 2023). Download Java for Windows. Abgerufen am 20. 02 2023 von java.com: https://www.java.com/download/ie_manual.jsp

Pomer, L. (03. 08 2022). Textklassifikation – Vorverarbeitung der Daten. Abgerufen am 14. 02 2023 von thecattlecrew.net: https://thecattlecrew.net/2022/08/03/textklassifikation-vorverarbeitung-der-daten/

Preusler, S. (13. 03 2020). Pyspark und Jupyter Notebook Anleitung für Windows. Abgerufen am 14. 01 2023 von https://medium.com/@stefan.preusler/pyspark-und-jupyter-notebook-anleitung-f%C3%BCr-windows-7317f2c968c4

Python Software Foundation. (2023). re — Regular expression operations¶. Abgerufen am 08. 03 2023 von docs.python.org: https://docs.python.org/3/library/re.html

Shah, A., 2018. How to setup Apache Spark(PySpark) on Jupyter/IPython Notebook?. [Online] 
Available at: https://medium.com/@ashish1512/how-to-setup-apache-spark-pyspark-on-jupyter-ipython-notebook-3330543ab307
[Accessed 14 01 2023].

Sarkar, T. (12. 11 2018). How to set up PySpark for your Jupyter notebook. Abgerufen am 14. 01 2023 von opensource.com: https://opensource.com/article/18/11/pyspark-jupyter-notebook

SparkBy{Examples}. (06. 12 2020). PySpark Liest JSON-Datei in DataFrame. Abgerufen am 14. 01 2023 von https://sparkbyexamples.com/pyspark/pyspark-read-json-file-into-dataframe/#read-json-multiline

Teja, S. (10. 06 2020). Stop Words in NLP. Abgerufen am 24. 02 2023 von medium.com: https://medium.com/@saitejaponugoti/stop-words-in-nlp-5b248dadad47

Wuttke, L. (2022). Einführung in Apache Spark: Komponenten, Vorteile und Anwendungsbereiche. Abgerufen am 05. 01 2023 von datasolut.com: https://datasolut.com/was-ist-spark/#was-ist-pyspark

Yuzhe-Lu. (10. 12 2018). Topic-Modeling-on-Amazon-Reviews-using-LDA/2_LDA_Data_Processing.ipynb. Abgerufen am 14. 02 2023 von https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb